In [1]:
# API KEY: sk-82F0fUi3hSi_drnaRG9Z72jEbFbIqPdYHHClASd7kET3BlbkFJJqxRZ40hP-p3XxjBMCQK3qUT40AmziQLi9Vz5WIcIA
# ORG ID: org-rjJCwF3sTsfOnw3I2Eaj4zdP
import json
import re
import math
import pandas as pd
from algorithms import *
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

#### Necessary pip installation commands include:
- pip install pandas
- pip install nltk
#### <b>Note</b>: have a <b>.env</b> file already created for accessing API key


In [2]:
def combined_randomized_preprocessing(data_one, data_two):
    data_one_pre = toxicity_challenge_preprocess(data_one)
    data_two_pre = toxicity_challenge_preprocess(data_two)
    print(f"Toxic: {len(data_one_pre[data_one_pre['Toxic'] == 1])}")
    print(f"Non-Toxic: {len(data_one_pre[data_two_pre['Toxic'] == 0])}")
    complete_data = pd.concat([data_one_pre, data_two_pre]).sample(frac=1, random_state=42).reset_index(drop=True)

    return complete_data

In [3]:
# reads kaggle toxic classification challenge dataset
df = pd.read_csv('../data/csv/train.csv')
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\r\nWhy the edits made under my use...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\r\nMore\r\nI can't make any real suggestions...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \r\n\r\nThat...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \r\n\r\nUmm, theres no actual article ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [4]:
len(df[df['toxic'] == 1])

15294

In [5]:
len(df[df['toxic'] == 0])

144277

In [6]:
df.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
training_size = 5000
evaluation_size = 500
container = 1500
toxic_set = df[df['toxic'] == 1 &
    ~df['comment_text'].apply(check_repetition)
].sample(n=(training_size + container),random_state=43)
non_toxic_set = df[df['toxic'] == 0 &
    ~df['comment_text'].apply(check_repetition)
].sample(n=(training_size + container),random_state=43)

In [8]:
toxic_train, toxic_eval = toxic_set[:training_size], toxic_set[training_size:].sample(frac=1, random_state=43).reset_index(drop=True)
non_toxic_train, non_toxic_eval = non_toxic_set[:training_size], non_toxic_set[training_size:].sample(frac=1, random_state=44).reset_index(drop=True)

In [9]:
toxic_eval[toxic_eval['toxic'] == 1].iloc[:500]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,e762800e2594e08c,"Album versions are notable, because otherwise ...",1,0,0,0,0,0
1,e712b9dbec73f4dd,This page is fucked up can someone please fix ...,1,0,1,0,0,0
2,6d8a0fb2ab9fe84b,"""\r\n| """"ugly comments"""" are a crime? Attackin...",1,0,0,0,1,0
3,ebe87453f8622e93,Imperialist bastards... \r\n\r\n...,1,0,1,0,1,0
4,fc90f69c57aabad5,REDIRECT Talk:The Liar the Bitch and Her Wardrobe,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...
575,1c48de3a7a8e08f7,Coward \r\n\r\nJust like a Bangladeshi. Couldn...,1,0,0,0,0,0
576,11cb3112bac94e0d,", more like gayfagonaplane",1,0,0,0,0,0
577,5630d385a498b686,"""\r\n\r\n HA! \r\n\r\nyou are gay!, what the f...",1,0,1,0,1,1
578,49d78b1a037f8564,Faggot \r\n\r\nQuit fucking wit mii sht niqqa,1,0,1,0,1,1


In [10]:
non_toxic_eval[non_toxic_eval['toxic'] == 0].iloc[:500]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,722f0228b393831c,November 2010 \r\n Please stop assuming owners...,0,0,0,0,0,0
1,4dab3a8a648f6d34,Please rethink your action,0,0,0,0,0,0
2,3c9adddf2d64b7c8,Thank you for reacting. At this time two edito...,0,0,0,0,0,0
3,477875a62a6eab1d,It looks like I got confused. Your energy calc...,0,0,0,0,0,0
4,31ad53bce2cb1e6d,"""\r\n\r\nIn computing, input/output, or I/O, r...",0,0,1,0,0,0
...,...,...,...,...,...,...,...,...
495,2b6efcf7aa11961e,"""\r\n\r\nPlease remember to sign all of your p...",0,0,0,0,0,0
496,9a14878f4e1d3ebe,Not controversial. He is a convicted felon for...,0,0,0,0,0,0
497,631a44f2bc98380e,Thank you for experimenting with Wikipedia. Y...,0,0,0,0,0,0
498,049f8167c0ce7ea2,"""\r\n1) Organisations also count as things.\r\...",0,0,0,0,0,0


In [11]:
toxic_train[toxic_train['toxic'] == 1].iloc[:2500]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
2468,0695e0af39cb0e22,Piss off you sanctimonious weed. You forget th...,1,0,0,0,0,0
110363,4e63286038b2bae2,Who the fuck is saying ths to me...? How is an...,1,0,1,0,1,0
23736,3eb303deae78cfb7,You are kidding me ? Look at the previous vers...,1,0,0,0,0,0
142751,fb7a291996b03b2c,", I just want to be left alone can't you fucki...",1,0,0,0,0,0
71472,bf4c77aa00ed3fd2,""":: Actually, it is you who """"resorted to path...",1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
72546,c222814faeed6b2b,Douche bag \r\n\r\nbro chill out with the vand...,1,0,1,0,0,0
142323,f93f73fdf97bb559,Thanks \r\n\r\nYou Chasewc91 & Nightscream mak...,1,0,1,0,1,0
119719,80249296a4bddd86,Is a complete idiot.,1,0,0,0,1,0
116761,702f8383f5de660d,unblock|Material scientist isa dum dum and so ...,1,0,0,0,0,0


In [12]:
non_toxic_train[non_toxic_train['toxic'] == 0].iloc[:2500]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
22283,3ac6d4820819d025,"Dear Lightocha, your claims of Buddhist origin...",0,0,0,0,0,0
147731,40e88ba1b4a86e9d,Accomplishments\r\n\r\nThis is where I'll post...,0,0,0,0,0,0
9014,17f9a977a695a706,Sheiks of Sheba attempt at deletion\r\n\r\nI d...,0,0,0,0,0,0
58958,9de2e518d3388a71,"Ah, par for the course - comments that are not...",0,0,0,0,0,0
44586,7722df898e8093fb,"Eenemies of Batman\r\nOK, I have explained my ...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
87789,ead016df297e63a0,Mentioning of over seas distribution ==\r\n\r\...,0,0,0,0,0,0
2325,0641f4f131492679,"""::::Narking - ok and that disproves my point ...",0,0,0,0,0,0
104578,2f7c6cd1cde25264,Thank you. Because I believe that the accusat...,0,0,0,0,0,0
120074,821d637d814fe418,Thanks \r\nI was just about to try reverting t...,0,0,1,0,1,0


In [13]:
full_training_set = combined_randomized_preprocessing(toxic_train[toxic_train['toxic'] == 1].iloc[:2500], non_toxic_train[non_toxic_train['toxic'] == 0].iloc[:2500])
full_evaluation_set = combined_randomized_preprocessing(toxic_eval[toxic_eval['toxic'] == 1].iloc[:500], non_toxic_eval[non_toxic_eval['toxic'] == 0].iloc[:500])


Toxic: 2500
Non-Toxic: 2500
Toxic: 500
Non-Toxic: 500


In [14]:
full_evaluation_set

,Text,Toxic
0,Date When did this algorithm get invented ?,0
1,answered them many times ! but they didnt answ...,0
2,IAC There is a discussion on article 's talk p...,0
3,Spandam 's laugh ? I 'm listening to it and al...,0
4,AND HAVING A FUCKIN ' BITCH ASS DAY '' ',1
...,...,...
995,"Oh , well Harry finally got his payback on me ...",1
996,this page is about random crap talk .. just ed...,1
997,I 've reverted the editsBOTH OF THEM VIOLATED ...,0
998,"I used the exact same bullshit you wrote , hyp...",1


In [15]:
len(full_evaluation_set[full_evaluation_set['Toxic'] == 1])

500

In [16]:
len(full_evaluation_set[full_evaluation_set['Toxic'] == 0])

500

In [17]:
len(full_training_set[full_training_set['Toxic'] == 0])

2500

In [18]:
len(full_training_set[full_training_set['Toxic'] == 1])

2500

In [19]:
# saves preprocessed fine_tuning data for later usage
# full_training_set.to_csv('../data/csv/toxic_classification_training_set.csv', index=False)
full_training_set.to_csv('../data/csv/semi_toxic_classification_training_set.csv', index=False)

In [20]:
# saves preprocessed evaluation data to csv file
full_evaluation_set.to_csv('../data/csv/semi_toxic_classification_evaluation_set.csv', index=False)